In [44]:
import re
from nltk.tokenize import regexp_tokenize
from os import listdir
from struct import pack, unpack

def ngram(s, n=2):
    rst = list()
    for i in range(len(s)-(n-1)):
        rst.append(''.join(s[i:i+n]))
    return rst

def fileids(path, ext='txt'):
    files = list(filter(lambda f:re.search(f'{ext}$', f),
                       listdir(path)))
    return list(map(lambda f:f'{path}/{f}', files))

p1 = re.compile(r'\s+')

D = list() # {file:문서1, maxfreq:뭐고}, 문서, ...
V = list()
TDM = dict() # {term1:[위치, df]}

Posting = open('posting.dat', 'wb')
Posting.close()

for file in fileids('naver'):
    i = len(D)
    D.append({'filename':file, 'maxfreq':0, 'length':0.0})
    
    with open(file, 'r', encoding='utf8') as fp:
        corpus = fp.read()
        
    localTDM = dict()
    localPosting = open('local.dat', 'wb')

    # Tokenizing+Normalizing
    for t in regexp_tokenize(p1.sub(' ', corpus), r'\b\w+\b'):
        for g in ngram(t):
            # 문서 1개 작업하는 중에, 단어가 처음 등장
            if g not in localTDM.keys():
                pos = localPosting.tell()
                localPosting.write(pack('ii', 1, -1))
                localTDM[g] = pos
            else:
                pos = localPosting.tell()
                localPosting.write(pack('ii', 1, localTDM[g]))
                localTDM[g] = pos
                
    localPosting.close()
                
    # Update; Local -> Global
    Posting = open('posting.dat', 'ab')
    localPosting = open('local.dat', 'rb')
    
    # k=단어, v=파일위치
    maxFreq = 0
    for k, v in localTDM.items():
        if k not in V:
            V.append(k)
            
        j = V.index(k)
        
        freq = 0
        pos = v
        while pos != -1:
            localPosting.seek(pos)
            f, npos = unpack('ii', localPosting.read(8))
            pos = npos
            freq += 1
            
        if freq > maxFreq:
            maxFreq = freq
                
        if j not in TDM.keys():
            pos = Posting.tell()
            Posting.write(pack('iii', i, freq, -1))
            TDM[j] = {'fp':pos, 'df':1}
        else:
            pos = Posting.tell()
            Posting.write(pack('iii', i, freq, TDM[j]['fp']))
            TDM[j]['fp'] = pos
            TDM[j]['df'] += 1
            
    D[i]['maxfreq'] = maxFreq

    localPosting.close()
    Posting.close()

In [45]:
i2w = lambda i:V[i]
w2i = lambda w:V.index(w)

i2d = lambda i:D[i]
d2i = lambda d:D.index(d)

In [46]:
from math import log

tf1 = lambda tf:1 if tf > 0 else 0
tf2 = lambda tf:tf
tf3 = lambda tf, ttf:tf/ttf
tf4 = lambda tf:log(1+tf)
tf5 = lambda tf, mtf, K=0.5:K+(1-K)*(tf/mtf)

idf1 = lambda df:1
idf2 = lambda df, N:log(N/df)
idf3 = lambda df, N:log(N/(1+df))+1
idf4 = lambda df, mdf:log(mdf/(1+df))
idf5 = lambda df, N:log((N-df)/df)

In [47]:
WDM = dict()
Posting = open('posting.dat', 'rb')
Weighting = open('weighting.dat', 'wb')

for k, v in TDM.items():
    fp = Weighting.tell()
    WDM[k] = {'fp':fp, 'df':v['df']}
    
    pos = v['fp']
    while pos != -1:
        Posting.seek(pos)
        i, freq, npos = unpack('iii', Posting.read(12))
        
        tf = tf5(freq, D[i]['maxfreq'], 0)
        idf = idf2(v['df'], len(D))
        w = tf*idf
        D[i]['length'] += w**2
        Weighting.write(pack('if', i, w))
        
        pos = npos
            
Posting.close()
Weighting.close()

In [48]:
query = '전손차량 중 확인되지 않는 차량이 4만여대에 달한다는 지적이 나왔다. 전손차량은 자동차가 완전히 파손됐거나 침수 등으로 수리할 수 없는 상태인 자동차와 발생한 손해액이 보험가액 이상인 자동차를 말한다.'

# Query Weighting
QTM = dict()

for q in regexp_tokenize(query, r'\b\w+\b'):
    for g in ngram(q): # ngram => tokenizing
        if g in V:
            j = w2i(g)
            if j not in QTM.keys():
                QTM[j] = 0
            QTM[j] += 1
            
for k,v in QTM.items():
    tf = tf5(v, max(QTM.values()), 0)
    idf = idf2(TDM[k]['df'], len(D))
    QTM[k] = tf*idf
    
# Distance
result = dict()
Weighting = open('weighting.dat', 'rb')

for t in V:
    j = w2i(t)
    
    qw = QTM.get(j, 0)
    n = 0
    while n < WDM[j]['df']:
        Weighting.seek(WDM[j]['fp']+n*8)
        i, dw = unpack('if', Weighting.read(8))
        pos = npos
        n += 1
        
        if i not in result.keys():
            result[i] = {'dist':0.0, 'cnt':0}
            
        result[i]['cnt'] += 1
        result[i]['dist'] += (qw-dw)**2
    
Weighting.close()

In [49]:
sorted(result.items(), key=lambda r:r[1]['dist'])[:10]

[(150, {'dist': 12.477792076118877, 'cnt': 1021}),
 (85, {'dist': 13.602877702605353, 'cnt': 628}),
 (84, {'dist': 17.626246390390296, 'cnt': 622}),
 (88, {'dist': 20.463704604943832, 'cnt': 604}),
 (206, {'dist': 21.47970325384423, 'cnt': 1481}),
 (348, {'dist': 22.132515351856252, 'cnt': 353}),
 (83, {'dist': 22.703285215299353, 'cnt': 641}),
 (45, {'dist': 27.62123876701429, 'cnt': 883}),
 (243, {'dist': 28.161195300115445, 'cnt': 657}),
 (308, {'dist': 28.784139539195497, 'cnt': 162})]

In [50]:
sorted(result.items(), key=lambda r:r[1]['dist'], reverse=True)[:10]

[(54, {'dist': 350.4286582619124, 'cnt': 256}),
 (321, {'dist': 340.58596350730437, 'cnt': 243}),
 (360, {'dist': 332.23415792698154, 'cnt': 95}),
 (356, {'dist': 306.40029529272033, 'cnt': 633}),
 (52, {'dist': 281.07616406652653, 'cnt': 264}),
 (165, {'dist': 267.8058227291391, 'cnt': 219}),
 (211, {'dist': 256.97845924989787, 'cnt': 411}),
 (67, {'dist': 252.79960945506326, 'cnt': 431}),
 (163, {'dist': 249.40806888739272, 'cnt': 199}),
 (96, {'dist': 245.6643303010328, 'cnt': 523})]

In [51]:
# query = '전손차량 중 확인되지 않는 차량이 4만여대에 달한다는 지적이 나왔다. 전손차량은 자동차가 완전히 파손됐거나 침수 등으로 수리할 수 없는 상태인 자동차와 발생한 손해액이 보험가액 이상인 자동차를 말한다.'
with open(D[22]['filename'], 'r', encoding='utf8') as f:
    query = p1.sub(' ', f.read())

# Query Weighting
QTM = dict()

for q in regexp_tokenize(query, r'\b\w+\b'):
    for g in ngram(q): # ngram => tokenizing
        if g in V:
            j = w2i(g)
            if j not in QTM.keys():
                QTM[j] = 0
            QTM[j] += 1
            
for k,v in QTM.items():
    tf = tf5(v, max(QTM.values()), 0)
    idf = idf2(TDM[k]['df'], len(D))
    QTM[k] = tf*idf

ql = sum([v**2 for k, v in QTM.items()])**(1/2)
for k,v in QTM.items():
    QTM[k] = v
    
# Angle
result = dict()
Weighting = open('weighting.dat', 'rb')

for k, v in QTM.items():
    j = k
    
    n = 0
    while n < WDM[j]['df']:
        Weighting.seek(WDM[j]['fp']+n*8)
        i, dw = unpack('if', Weighting.read(8))
        pos = npos
        n += 1
        
        if i not in result.keys():
            result[i] = {'dist':0.0, 'cnt':0}
            
        result[i]['cnt'] += 1
        result[i]['dist'] += v*dw # 내적

for k, v in result.items(): # 정규화
    result[k]['dist'] /= ql * D[k]['length']**(1/2)
    #                    쿼리길이      다큐먼트길이
    
Weighting.close()

In [52]:
sorted(result.items(), key=lambda r:r[1]['dist'], reverse=True)[:10]

[(22, {'dist': 0.9397504593905724, 'cnt': 168}),
 (89, {'dist': 0.4060440391097903, 'cnt': 79}),
 (76, {'dist': 0.36888435213755366, 'cnt': 79}),
 (37, {'dist': 0.3641186046231235, 'cnt': 61}),
 (315, {'dist': 0.25947112604165945, 'cnt': 62}),
 (59, {'dist': 0.2523920457005235, 'cnt': 61}),
 (232, {'dist': 0.19459779266653335, 'cnt': 62}),
 (34, {'dist': 0.13987475721531556, 'cnt': 38}),
 (60, {'dist': 0.1372369108149422, 'cnt': 32}),
 (85, {'dist': 0.09548805435195587, 'cnt': 25})]

In [53]:
sorted(result.items(), key=lambda r:r[1]['dist'])[:10]

[(125, {'dist': 0.0001651989406841924, 'cnt': 1}),
 (177, {'dist': 0.00039167303195381805, 'cnt': 5}),
 (331, {'dist': 0.0006214405081663085, 'cnt': 3}),
 (323, {'dist': 0.0008902586480422132, 'cnt': 4}),
 (116, {'dist': 0.0008999992092287987, 'cnt': 4}),
 (378, {'dist': 0.001146442570330567, 'cnt': 9}),
 (250, {'dist': 0.0012972282319964764, 'cnt': 9}),
 (290, {'dist': 0.001428416328520135, 'cnt': 7}),
 (229, {'dist': 0.0014584330771862323, 'cnt': 8}),
 (314, {'dist': 0.0015138025755681585, 'cnt': 8})]

In [55]:
with open(D[0]['filename'], 'r', encoding='utf8') as f:
    print(f.read())

















 ◀앵커▶장수군의 관광 명소로 떠오르고 있는 누리파크에 이색적인 분위기의 감성 정원이 조성됐습니다.완주 운주면에 아홉 번째 작은 도서관이 문을 열었습니다.우리 지역 소식 허현호 기자입니다.◀리포트▶[장수]장수군이 누리파크 일원에 유럽형 가족정원을 조성해 관람객들의 발길이 이어지고 있습니다.'한국의 타샤 튜더'로 불리는 임지수 정원가드너가 총괄한 가족정원은 연꽃정원과 맨발정원, 물빛정원 등을 테마로 희귀한 꽃과 이국적인 나무가 심어져 있습니다.또 가을을 맞아 국화꽃길과 웨딩정원도 마련돼 색다른 볼거리를 제공하고 있습니다.[최석원 / 장수군 산림공원과 산림정책팀장]"누리파크를 아이들과 어른들이 함께할 수 있는 이색적인 가족정원으로 만들어 대한민국을 대표하는 관광지로 만들어나가겠습니다."[완주]완주 운주면행정복지센터에 '구름골 작은도서관'이 개관했습니다.운주면민들의 문화사랑방이 될 도서관은 2천200여권의 장서가 비치됐고, 소모임 활동과 책 읽기, 독서 프로그램이 가능한 공간으로 꾸며졌습니다.완주군은 누구나 도서관 문화를 누릴 수 있도록 작은도서관을 조성해 현재 8개 읍·면에서 운영하고 있습니다.[김미경 / 완주군도서관사업소 도서관정책팀장]"작은도서관이 우리 아이들의 꿈과 희망이 넘치는 곳이고, 어르신들이 정을 나누는 행복한 공간이 됐으면 좋겠습니다."[남원]전라북도에서 추진하는 농공단지 환경개선 공모사업에 남원시가 선정돼 총사업비 2억 원을 확보했습니다.남원시는 노암농공단지 구내식당 환경개선과 함께 근로자 편의 공간과 휴게실을 확충하는 등 일하기 좋은 환경을 조성해 지역 청년과 노동자들을 유입시킬 계획입니다.[익산]익산시가 세계문화유산과 야간경관 콘텐츠 등 지역 관광자원을 활용해 국외 관광객 유치에 나섭니다.익산시는 중국과 대만, 홍콩 등 중화권을 전담하는 여행사와 공동마케팅 업무협약을 맺고, 특수목적 관광객과 수학여행단 등 외래 관광객 유치를 위한 관광상품을 개발하고 홍보할 방침입니다.MBC뉴스 허현호입니다.영상편집:김관중영상제공:장

In [63]:
#전처리 기를 하나 만들어서 걸어야 할 표현 거르기
p1 = re.compile(r'\s+')  #위아래 공백 없애기
p2 = re.compile(r'^\s+')
p3 = re.compile(r'\s+$')
p4 = re.compile(r'[^0-9a-zA-Zㄱ-ㅎㅏ-ㅣ가-힣.,% ]')  #구두점 날리기

def preprocessing(file):
    with open(file, 'r', encoding='utf8') as fp:
        doc = fp.read()
        print(p3.sub(' ', (p2.sub(' ', (p1.sub(' ', p4.sub(' ', doc)))))))
        #순서에 따라 결과 달라짐. 

preprocessing(D[0]['filename'])

 앵커 장수군의 관광 명소로 떠오르고 있는 누리파크에 이색적인 분위기의 감성 정원이 조성됐습니다.완주 운주면에 아홉 번째 작은 도서관이 문을 열었습니다.우리 지역 소식 허현호 기자입니다. 리포트 장수 장수군이 누리파크 일원에 유럽형 가족정원을 조성해 관람객들의 발길이 이어지고 있습니다. 한국의 타샤 튜더 로 불리는 임지수 정원가드너가 총괄한 가족정원은 연꽃정원과 맨발정원, 물빛정원 등을 테마로 희귀한 꽃과 이국적인 나무가 심어져 있습니다.또 가을을 맞아 국화꽃길과 웨딩정원도 마련돼 색다른 볼거리를 제공하고 있습니다. 최석원 장수군 산림공원과 산림정책팀장 누리파크를 아이들과 어른들이 함께할 수 있는 이색적인 가족정원으로 만들어 대한민국을 대표하는 관광지로 만들어나가겠습니다. 완주 완주 운주면행정복지센터에 구름골 작은도서관 이 개관했습니다.운주면민들의 문화사랑방이 될 도서관은 2천200여권의 장서가 비치됐고, 소모임 활동과 책 읽기, 독서 프로그램이 가능한 공간으로 꾸며졌습니다.완주군은 누구나 도서관 문화를 누릴 수 있도록 작은도서관을 조성해 현재 8개 읍 면에서 운영하고 있습니다. 김미경 완주군도서관사업소 도서관정책팀장 작은도서관이 우리 아이들의 꿈과 희망이 넘치는 곳이고, 어르신들이 정을 나누는 행복한 공간이 됐으면 좋겠습니다. 남원 전라북도에서 추진하는 농공단지 환경개선 공모사업에 남원시가 선정돼 총사업비 2억 원을 확보했습니다.남원시는 노암농공단지 구내식당 환경개선과 함께 근로자 편의 공간과 휴게실을 확충하는 등 일하기 좋은 환경을 조성해 지역 청년과 노동자들을 유입시킬 계획입니다. 익산 익산시가 세계문화유산과 야간경관 콘텐츠 등 지역 관광자원을 활용해 국외 관광객 유치에 나섭니다.익산시는 중국과 대만, 홍콩 등 중화권을 전담하는 여행사와 공동마케팅 업무협약을 맺고, 특수목적 관광객과 수학여행단 등 외래 관광객 유치를 위한 관광상품을 개발하고 홍보할 방침입니다.MBC뉴스 허현호입니다.영상편집 김관중영상제공 장수군 최민광 , 완주군 김회성 , 남원시 강석현 

In [61]:
#위 아래 공백이 사라진걸 볼 수 있다. 쓸데없는 기호도 사라진게 보인다. 

In [68]:
#내 데이터에 따라서 전처리 patterns 만들어주고 적용하기

p1 = re.compile(r'\s+')  #위아래 공백 없애기
p2 = re.compile(r'^\s+')
p3 = re.compile(r'\s+$')
p4 = re.compile(r'[^0-9a-zA-Zㄱ-ㅎㅏ-ㅣ가-힣.,% ]')  #구두점 날리기

def preprocessing(file):
    with open(file, 'r', encoding='utf8') as fp:
        doc = p3.sub(' ',
              p2.sub(' ',
              p1.sub(' ',
              p4.sub(' ', fp.read()))))
    return doc

In [98]:
from nltk.tokenize import word_tokenize, sent_tokenize
from konlpy.tag import Okt

ma = Okt()

def tokenizing(doc):
    term = list()
    
    for s in sent_tokenize(doc):
        s1= word_tokenize(s)
        s2= ma.morphs(s)


        #2음절씩 보기
        for g in ngram(s):
            term.append(g)
            
        for t in s1:
            term.append(t)

        for g in ngram(s1):
            term.append(g)
            
        #형태소 분석한것도 쓰기
        for t in s2:
            term.append(t)

        for g in ngram(s2):
            term.append(g)

    return term


In [99]:
tokenizing(preprocessing(D[0]['filename']))

KeyError: 'filename'

In [100]:
#색인기 만들기
from collections import Counter

def indexer(tokens):
    localTDM = Counter(tokens)
    #Counter 로 처리하면 키 밸류 쌍으로 처리가 된다.
    return localTDM, max(localTDM.values()) #나중에 정규화를 위해 이 값들이 필요한거다.
    

In [105]:
D = list()
V = list()
TDM = dict()

#Posting file 관련된 것들
posting = 'posting.dat'

#훼이크로 빈파일 생성하고 닫기
open(posting, 'wb').close()

for file in fileids('naver')[:100]:
    i = len(D)
    localTDM, maxFreq = indexer(tokenizing(preprocessing(file)))
    D.append({'file':file, 'mf': maxFreq, 'dl': 0.0})

#원래는 여기서 merge 를 해줘야함.
    #merge 부분에서 posting 열기
    fp = open(posting, 'ab')
    #merge 에서는 localTDM 을 키 밸류로 돌릴거다.
    for k, v in localTDM.items():
        if k not in V:
            V.append(k)
        j = w2i(k)

        if j not in TDM:
            #j에 안들어있으면, 위치가 지금 어딘지, document frequency 가 어떻게 되는지
            TDM[j] = {'pos1':-1,'pos2':-1, 'df':0}

        #file 에다가 기록하기
        pos = fp.tell() #현재 위치 알려줘
        fp.write(pack('iii', i, v, TDM[j]['pos1']))  #어떤문서에서 몇번 나왔고 마지막 위치 (pos : -1)
        TDM[j]['pos1'] = pos
        TDM[j]['df'] += 1
    fp.close()

KeyboardInterrupt: 

In [77]:
#위 코드 전체가 색인하는 과정이다. 

### weight 주기

In [103]:
weighting = 'weighting.dat'

#똑같이 TDM 열기
fp1 = open(posting, 'rb')
fp2 = open(weighting, 'wb')

for k, v in TDM.items():
    j = k

    pos1 = v['pos1']
    pos2 = fp2.tell()
    while pos1 != -1:
        fp1.seek(pos1)
        i, freq, npos1 = unpack('iii', fp1.read(12))
        pos1 = npos1

        #weight 주기
        w = tf5(freq, D[i]['mf'],0) * idf2(v['df'], len(D))
        fp2.write(pack('if', i, w))

        #D에 기록하기
        D[i]['dl'] += w**2
    #weight 기록하기
    TDM[j]['pos2'] = pos2
    
fp1.close()
fp2.close()

IndexError: list index out of range

### query 를 가지고 해보기 

In [104]:
Q = fileids('naver')[10]

tokens, maxFreq = indexer(tokenizing(preprocessing(Q)))

QTM = dict()
for k, v in tokens.items():
    j = w2i(k)
    w = tf5(v, maxFreq, 0) * idf2(TDM[j]['df'], len(D))
    QTM[j] = w 

q1 = sum([w**2 for w in QTM.values()])**.5

ValueError: '중위' is not in list

In [ ]:
result = dict()

fp = open(weighting, 'rb')

for j, qw in QTM.items():
    n = 0
    pos = TDM[j]['pos2']
    while n < TDM[j]['df']:
        fp.seek(pos+n*8)
        i, tw = unpack('if', fp.read(8))
        n +=1

        if i not in result:
            result[i] = .0
            
        result[i] += qw * tw

for k, v in result.items():
    # result[k] = v / (ql * (D[k]['dl']**.5))
    result[k] = v / (D[k]['dl']**.5)

fp.close()

In [ ]:
sorted(result.items(), key=lambda r:r[1], reverse=True)[:10]

In [ ]:
for i, sim in sorted(result.items(),
                     key=lambda r:r[1], reverse=True)[:10]
    print(D[i]['file'], sim)
    print(preprocessing(D[i]['file'])[:100])
    print()

In [ ]:
#여기까지가 우리가 아는 검색 방법이라고 한다. sorting 에 따라서 구글이 되기도 네이버가 되기도 함. 
#저러한 색인어들이 꼭 필요한건 아니다. 
#classification, clustering - 적은 컬럼이 

### 저장하기

In [ ]:
import json

with open('D.dat', 'w', encoding='utf8') as fp:
    json.dump(D, fp)

with open('V.dat', 'w', encoding='utf8') as fp:
    json.dump(V, fp)

with open('TDM.dat', 'w', encoding='utf8') as fp:
    json.dump(TDM, fp)

In [ ]:
!dir *.dat

## 수집기

In [ ]:
# 네이버 뉴스 수집기 - Crawler
from requests import get
from bs4 import BeautifulSoup
import re

urls = list()
seens = list()

path = 'naver/'
urls.append('https://news.naver.com/')

headers = {'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36'}

while urls:
    url = urls.pop(0)
    seens.append(url)
    
    dom = BeautifulSoup(get(url,headers=headers).text, 'html.parser')
    
    # 메뉴
    alist = dom.select('ul[role=menu] > li > a[href]')
    for a in alist:
        if re.search(r'sid1=\d{3}$', a.attrs['href']):
            if a.attrs['href'] not in urls and\
               a.attrs['href'] not in seens:
                urls.append(a.attrs['href'])
                
    # 기사링크 => 정교하게
    alist = dom.select('''
                        .sh_text > a[href],
                        .cluster_text > a[href],
                        dt > a[href]
                    ''')
    for a in alist:
        if a.attrs['href'] not in urls and\
           a.attrs['href'] not in seens:
            urls.append(a.attrs['href'])
            
    # 본문
    news = dom.select_one('#newsct_article')
    if news:
        # https://n.news.naver.com/article/005/0001644886?cds=news_media_pc
        file = '-'.join(
            re.search(r'(\d{10})\?sid=(\d{3})', url).groups())
        with open(f'{path}{file}.txt', 'w', encoding='utf-8') as fp:
            fp.write(news.text)

In [ ]:
import re

## load 하기

In [ ]:
import json

with open('D.dat', encoding='utf8') as fp:
    D = json.load(fp)

with open('V.dat', encoding='utf8') as fp:
    V = json.load(fp)

with open('TDM.dat', encoding='utf8') as fp:
    TDM = json.load(fp)

### dynamic 으로 다시 검색을 해서, KNN 이용...

In [ ]:
p1 = re.compile(r'\s+')  #위아래 공백 없애기
p2 = re.compile(r'^\s+')
p3 = re.compile(r'\s+$')
p4 = re.compile(r'[^0-9a-zA-Zㄱ-ㅎㅏ-ㅣ가-힣.,% ]')  #구두점 날리기

def preprocessing(file):
    with open(file, 'r', encoding='utf8') as fp:
        doc = p3.sub(' ',
              p2.sub(' ',
              p1.sub(' ',
              p4.sub(' ', fp.read()))))
    return doc

In [ ]:
#색인기 만들기
from collections import Counter

def indexer(tokens):
    localTDM = Counter(tokens)
    #Counter 로 처리하면 키 밸류 쌍으로 처리가 된다.
    return localTDM, max(localTDM.values()) #나중에 정규화를 위해 이 값들이 필요한거다.

In [114]:
len(D), len(V)

(77, 48844)

In [116]:
len(D), len(V)

(77, 48844)

In [115]:
posting = 'posting.dat'

for file in fileids('naver')[:10]: #시간 오래 걸려서 [:10] 으로 함. ...
    if len(list(filter(lambda r:r['file'] == file, D))) >0:
        continue
        
    i = len(D)
    localTDM, maxFreq = indexer(tokenizing(preprocessing(file)))
    D.append({'file':file, 'mf': maxFreq, 'dl': 0.0})

    fp = open(posting, 'ab')
    for k, v in localTDM.items():
        if k not in V:
            V.append(k)
        j = w2i(k)

        if j not in TDM:
            TDM[j] = {'pos1':-1,'pos2':-1, 'df':0}

        pos = fp.tell() #현재 위치 알려줘
        fp.write(pack('iii', i, v, TDM[j]['pos1']))  #어떤문서에서 몇번 나왔고 마지막 위치 (pos : -1)
        TDM[j]['pos1'] = pos
        TDM[j]['df'] += 1
    fp.close()

In [ ]:
import re
from nltk.tokenize import regexp_tokenize
from os import listdir
from struct import pack, unpack

def ngram(s, n=2):
    rst = list()
    for i in range(len(s)-(n-1)):
        rst.append(''.join(s[i:i+n]))
    return rst

def fileids(path, ext='txt'):
    files = list(filter(lambda f:re.search(f'{ext}$', f),
                       listdir(path)))
    return list(map(lambda f:f'{path}/{f}', files))

p1 = re.compile(r'\s+')

D = list() # {file:문서1, maxfreq:뭐고}, 문서, ...
V = list()
TDM = dict() # {term1:[위치, df]}

Posting = open('posting.dat', 'wb')
Posting.close()

for file in fileids('naver'):
    i = len(D)
    D.append({'filename':file, 'maxfreq':0, 'length':0.0})
    
    with open(file, 'r', encoding='utf8') as fp:
        corpus = fp.read()
        
    localTDM = dict()
    localPosting = open('local.dat', 'wb')

    # Tokenizing+Normalizing
    for t in regexp_tokenize(p1.sub(' ', corpus), r'\b\w+\b'):
        for g in ngram(t):
            # 문서 1개 작업하는 중에, 단어가 처음 등장
            if g not in localTDM.keys():
                pos = localPosting.tell()
                localPosting.write(pack('ii', 1, -1))
                localTDM[g] = pos
            else:
                pos = localPosting.tell()
                localPosting.write(pack('ii', 1, localTDM[g]))
                localTDM[g] = pos
                
    localPosting.close()
                
    # Update; Local -> Global
    Posting = open('posting.dat', 'ab')
    localPosting = open('local.dat', 'rb')
    
    # k=단어, v=파일위치
    maxFreq = 0
    for k, v in localTDM.items():
        if k not in V:
            V.append(k)
            
        j = V.index(k)
        
        freq = 0
        pos = v
        while pos != -1:
            localPosting.seek(pos)
            f, npos = unpack('ii', localPosting.read(8))
            pos = npos
            freq += 1
            
        if freq > maxFreq:
            maxFreq = freq
                
        if j not in TDM.keys():
            pos = Posting.tell()
            Posting.write(pack('iii', i, freq, -1))
            TDM[j] = {'fp':pos, 'df':1}
        else:
            pos = Posting.tell()
            Posting.write(pack('iii', i, freq, TDM[j]['fp']))
            TDM[j]['fp'] = pos
            TDM[j]['df'] += 1
            
    D[i]['maxfreq'] = maxFreq

    localPosting.close()
    Posting.close()

### weighting 해보기

In [119]:
weighting = 'weighting.dat'

#똑같이 TDM 열기
fp1 = open(posting, 'rb')
fp2 = open(weighting, 'wb')

for k, v in TDM.items():
    j = k

    pos1 = v['pos1']
    pos2 = fp2.tell()
    while pos1 != -1:
        fp1.seek(pos1)
        i, freq, npos1 = unpack('iii', fp1.read(12))
        pos1 = npos1

        #weight 주기
        w = tf5(freq, D[i]['mf'],0) * idf2(v['df'], len(D))
        fp2.write(pack('if', i, w))

        #D에 기록하기
        D[i]['dl'] += w**2
    #weight 기록하기
    TDM[j]['pos2'] = pos2
    
fp1.close()
fp2.close()

In [122]:
Q = fileids('naver')[60]

tokens, maxFreq = indexer(tokenizing(preprocessing(Q)))

QTM = dict()
for k, v in tokens.items():
    j = w2i(k)
    w = tf5(v, maxFreq, 0) * idf2(TDM[j]['df'], len(D))
    QTM[j] = w 

q1 = sum([w**2 for w in QTM.values()])**.5

In [123]:
V.index('이균'), len(TDM)

(18591, 48844)

In [ ]:
fp = open(weighting, 'rb')
pos = TDM[0]['pos2']

n = 0
while n < TDM[0]['df']:
    fp.seek(pos+n*8)
    print(unpack('if', fp.read(8)))
fp.close()

In [126]:
pos

0

In [ ]:
k = 10 


for i, sim in sorted(result.items(),
                     key=lambda r:r[1], reverse=True)[:10]
    print(D[i]['file'], sim)
    print(preprocessing(D[i]['file'])[:100])
    print()